# Исследование надежности заемщиков

## Откроем таблицу и изучим общую информацию о данных

In [1]:
import pandas as pd # импортируем библиотеку pandas

In [2]:
data = pd.read_csv('/datasets/data.csv') # прочитаем csv-файл

In [3]:
print(data.head(20))# Выведем первые 20 строчек датафрейма `data` на экран.

    children  days_employed  dob_years            education  education_id  \
0          1   -8437.673028         42               высшее             0   
1          1   -4024.803754         36              среднее             1   
2          0   -5623.422610         33              Среднее             1   
3          3   -4124.747207         32              среднее             1   
4          0  340266.072047         53              среднее             1   
5          0    -926.185831         27               высшее             0   
6          0   -2879.202052         43               высшее             0   
7          0    -152.779569         50              СРЕДНЕЕ             1   
8          2   -6929.865299         35               ВЫСШЕЕ             0   
9          0   -2188.756445         41              среднее             1   
10         2   -4171.483647         36               высшее             0   
11         0    -792.701887         40              среднее             1   

In [4]:
data.info() # Выведем основную информацию

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Предобработка данных

### Удаление пропусков

**Выведем количество пропущенных значений для каждого столбца с ипользованием двух методов.**

In [5]:
print(data.isna().sum())

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64


**В двух столбцах есть пропущенные значения. Один из них — `days_employed`. Пропуски в этом столбце обработаем на следующем этапе. Другой столбец с пропущенными значениями — `total_income` — хранит данные о доходах. На сумму дохода сильнее всего влияет тип занятости, поэтому заполним пропуски в этом столбце нужно медианным значением по каждому типу из столбца `income_type`.**

In [6]:
# найдем медиану дохода для каждого типа занятости income_type
median_total = data.groupby('income_type')['total_income'].median()
# print(median_total)
# безработный = 131339.751676

# циклом пройдемся по уникальным значениям столбца income_type {row = сотрудник, компаньон и т.д.}
for row in data['income_type'].unique():
    #найдем пропущенное значение в столбце дохода (data['total_income'].isna())
    data.loc[(data['total_income'].isna()) & (data['income_type'] == row), 'total_income'] = median_total[row]
    #присвоим ему медианное значение дохода соответствущего типа занятости  
print(data.isna().sum()) #проверим, сколько осталось отсутствующих значений в столбце total_income


children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income           0
purpose                0
dtype: int64


### Обработка аномальных значений

**В данных могут встречаться артефакты (аномалии) — значения, которые не отражают действительность и появились по какой-то ошибке. Таким артефактом будет отрицательное количество дней трудового стажа в столбце `days_employed`. Для реальных данных это нормально. Обработаем значения в этом столбце: заменим все отрицательные значения положительными с помощью метода `abs()`.**

In [7]:
data['days_employed'] = data['days_employed'].abs()
print(data['days_employed'].unique())

[8437.67302776 4024.80375385 5623.42261023 ... 2113.3468877  3112.4817052
 1984.50758853]


**Для каждого типа занятости выведем медианное значение трудового стажа `days_employed` в днях.**

In [8]:
print(data.groupby('income_type')['days_employed'].median())

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64


У двух типов (безработные и пенсионеры) получатся аномально большие значения. Исправить такие значения сложно, поэтому оставьте их как есть. Тем более этот столбец не понадобится вам для исследования.

**Выведем перечень уникальных значений столбца `children`.**

In [9]:
print(data['children'].unique())

[ 1  0  3  2 -1  4 20  5]


**В столбце `children` есть два аномальных значения. Удалим строки, в которых встречаются такие аномальные значения из датафрейма `data`.**

In [10]:
data = data.loc[(data['children'] != -1) & (data['children'] != 20)]

**Ещё раз выведем перечень уникальных значений столбца `children`, чтобы убедиться, что артефакты удалены.**

In [11]:
print(data['children'].unique())

[1 0 3 2 4 5]


### Удаление пропусков (продолжение)

**Заполним пропуски в столбце `days_employed` медианными значениями по каждому типу занятости `income_type`.**

In [12]:
# найдем медиану days_employed для каждого типа занятости income_type
median_days_employed = data.groupby('income_type')['days_employed'].median()
# print(median_days_employed)
# безработный = 366413.652744 income_type

# циклом пройдемся по уникальным значениям столбца days_employed 
for index in data['income_type'].unique():
    #найдем пропущенное значение в столбце days_employed
    data.loc[(data['days_employed'].isna()) & (data['income_type'] == index), 'days_employed'] = median_days_employed[index]
    #присвоим ему медианное значение дохода соответствущего типа занятости  
print(data.isna().sum()) #проверим, сколько осталось отсутствующих значений в столбце days_employed



children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64


**Убедимся, что все пропуски заполнены. Ещё раз выведем количество пропущенных значений для каждого столбца с помощью двух методов.**

In [13]:
print(data['days_employed'].isna().sum())

0


### Изменение типов данных

**Заменим вещественный тип данных в столбце `total_income` на целочисленный с помощью метода `astype()`.**

In [14]:
data['total_income']= data['total_income'].astype(int)

### Обработка дубликатов

**Обработаем неявные дубликаты в столбце `education`. В этом столбце есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв. Приведем их к нижнему регистру.**

In [15]:
data['education']=data['education'].str.lower()

**Выведем на экран количество строк-дубликатов в данных.**

In [16]:
# посчитаем дубликаты
print(data.duplicated().sum())

71


In [17]:
# удалим дубликаты
data = data.drop_duplicates()

### Категоризация данных

**На основании диапазонов, указанных ниже, создадим в датафрейме `data` столбец `total_income_category` с категориями:**

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.


**Пример: кредитополучателю с доходом 25000 нужно назначить категорию `'E'`, а клиенту, получающему 235000, — `'B'`.**

In [18]:
# создаем функцию categorize_income()
def categorize_income(index):
    if 0<= index <= 30000:
        return 'E'
    if 30000 <= index <= 50000:
        return 'D'
    if 50001 <= index <= 200000:
        return 'C'    
    if 200001 <= index <= 1000000:
        return 'B'
    if 1000001 <= index:
        return 'A'

In [19]:
# применим функцию методом apply()
data['total_income_category'] = data['total_income'].apply(categorize_income)

**Выведем на экран перечень уникальных целей взятия кредита из столбца `purpose`.**

In [20]:
print(data['purpose'].unique()) 

['покупка жилья' 'приобретение автомобиля' 'дополнительное образование'
 'сыграть свадьбу' 'операции с жильем' 'образование'
 'на проведение свадьбы' 'покупка жилья для семьи' 'покупка недвижимости'
 'покупка коммерческой недвижимости' 'покупка жилой недвижимости'
 'строительство собственной недвижимости' 'недвижимость'
 'строительство недвижимости' 'на покупку подержанного автомобиля'
 'на покупку своего автомобиля' 'операции с коммерческой недвижимостью'
 'строительство жилой недвижимости' 'жилье'
 'операции со своей недвижимостью' 'автомобили' 'заняться образованием'
 'сделка с подержанным автомобилем' 'получение образования' 'автомобиль'
 'свадьба' 'получение дополнительного образования' 'покупка своего жилья'
 'операции с недвижимостью' 'получение высшего образования'
 'свой автомобиль' 'сделка с автомобилем' 'профильное образование'
 'высшее образование' 'покупка жилья для сдачи' 'на покупку автомобиля'
 'ремонт жилью' 'заняться высшим образованием']


**Создадим функцию `categorize_purpose()`, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:**

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.

**Пример: если в столбце `purpose` находится подстрока `'на покупку автомобиля'`, то в столбце `purpose_category` должна появиться строка `'операции с автомобилем'`.**

In [21]:
# создайте функцию categorize_purpose()
# на вход подается столбец со строками 
def categorize_purpose(index):    
    if 'авто' in index:
        return 'операции с автомобилем'
    if  'недвиж' in index or 'жил' in index:
        return 'операции с недвижимостью'
    if 'свад' in index:
        return 'проведение свадьбы'
    if 'образ' in index:
        return 'получение образования'   


In [22]:
# примените функцию методом apply()
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

print(data.head(20))

    children  days_employed  dob_years            education  education_id  \
0          1    8437.673028         42               высшее             0   
1          1    4024.803754         36              среднее             1   
2          0    5623.422610         33              среднее             1   
3          3    4124.747207         32              среднее             1   
4          0  340266.072047         53              среднее             1   
5          0     926.185831         27               высшее             0   
6          0    2879.202052         43               высшее             0   
7          0     152.779569         50              среднее             1   
8          2    6929.865299         35               высшее             0   
9          0    2188.756445         41              среднее             1   
10         2    4171.483647         36               высшее             0   
11         0     792.701887         40              среднее             1   